In [1]:
"""
MODIFIED ONE - CLONED FROM THE ORIGINAL CODE
"""

'\nMODIFIED ONE - CLONED FROM THE ORIGINAL CODE\n'

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import warnings

warnings.filterwarnings("ignore")

In [2]:
def read_csv_point_cloud(csv_file):
    data = pd.read_csv(csv_file, header=None)
    return data

In [3]:
fileName = "L004"

In [4]:
csv_file = f"RawDatasets/Toronto/{fileName}.csv"
points = read_csv_point_cloud(csv_file)
points

,0
0,627574.11096191 4842233.14898682 137.71000671 ...
1,627574.23400879 4842233.19000244 136.30400085 ...
2,627574.17797852 4842233.31896973 136.17100525 ...
3,627574.16198730 4842233.01898193 137.47999573 ...
4,627574.14398193 4842233.17102051 137.34599304 ...
...,...
6747643,627641.79797363 4842072.69201660 137.09100342 ...
6747644,627641.78002930 4842072.74700928 137.03999329 ...
6747645,627641.74298096 4842072.80401611 136.98800659 ...
6747646,627641.69097900 4842072.91699219 136.88499451 ...


In [5]:
points[['x', 'y', 'z', 'R', 'G', 'B', 'label']] = points[0].str.split(' ', expand=True)
points

,0,x,y,z,R,G,B,label
0,627574.11096191 4842233.14898682 137.71000671 ...,627574.11096191,4842233.14898682,137.71000671,98,99,104,6.000000
1,627574.23400879 4842233.19000244 136.30400085 ...,627574.23400879,4842233.19000244,136.30400085,89,83,83,6.000000
2,627574.17797852 4842233.31896973 136.17100525 ...,627574.17797852,4842233.31896973,136.17100525,102,103,98,6.000000
3,627574.16198730 4842233.01898193 137.47999573 ...,627574.16198730,4842233.01898193,137.47999573,109,110,92,6.000000
4,627574.14398193 4842233.17102051 137.34599304 ...,627574.14398193,4842233.17102051,137.34599304,98,101,106,6.000000
...,...,...,...,...,...,...,...,...
6747643,627641.79797363 4842072.69201660 137.09100342 ...,627641.79797363,4842072.69201660,137.09100342,57,55,56,3.000000
6747644,627641.78002930 4842072.74700928 137.03999329 ...,627641.78002930,4842072.74700928,137.03999329,55,53,54,3.000000
6747645,627641.74298096 4842072.80401611 136.98800659 ...,627641.74298096,4842072.80401611,136.98800659,51,49,52,3.000000
6747646,627641.69097900 4842072.91699219 136.88499451 ...,627641.69097900,4842072.91699219,136.88499451,53,52,50,3.000000


In [6]:
"""
UTM_OFFSET = [627285, 4841948, 0]

We need to subtract these values from x,y,z
"""
cols_to_float = ['x', 'y', 'z', 'R', 'G', 'B']
points[cols_to_float] = points[cols_to_float].astype(float)
# points['label'] = points['label'].astype(int)


UTM_OFFSET = [627285, 4841948, 0]
points['x'] = points['x'] - UTM_OFFSET[0]
points['y'] = points['y'] - UTM_OFFSET[1]
points['z'] = points['z'] - UTM_OFFSET[2]

In [7]:
classes = points['label']

In [8]:
#points.drop(columns=[0, 'label'], inplace=True)
points.drop(columns=[0, 'label', 'R', 'G', 'B'], inplace=True)
points.to_csv('ProcessedData/points.csv')
#points = points.astype(float)

In [9]:
xyz = points.to_numpy()

In [10]:
xyz

array([[289.11096191, 285.14898682, 137.71000671],
       [289.23400879, 285.19000244, 136.30400085],
       [289.17797852, 285.31896973, 136.17100525],
       ...,
       [356.74298096, 124.80401611, 136.98800659],
       [356.690979  , 124.91699219, 136.88499451],
       [356.6920166 , 124.9699707 , 136.83500671]])

In [11]:
%%time

from jakteristics import las_utils, compute_features, FEATURE_NAMES
print(FEATURE_NAMES)
features = compute_features(xyz, search_radius=0.25, feature_names=['eigenvalue_sum', 'omnivariance', 'eigenentropy', 'anisotropy', 'planarity', 'linearity', 'PCA1', 'PCA2', 'surface_variation', 'sphericity', 'verticality', 'nx', 'ny', 'nz'])

['eigenvalue_sum', 'omnivariance', 'eigenentropy', 'anisotropy', 'planarity', 'linearity', 'PCA1', 'PCA2', 'surface_variation', 'sphericity', 'verticality', 'nx', 'ny', 'nz', 'number_of_neighbors', 'eigenvalue1', 'eigenvalue2', 'eigenvalue3', 'eigenvector1x', 'eigenvector1y', 'eigenvector1z', 'eigenvector2x', 'eigenvector2y', 'eigenvector2z', 'eigenvector3x', 'eigenvector3y', 'eigenvector3z']
CPU times: user 4min 36s, sys: 28min 23s, total: 33min
Wall time: 18min 34s


In [12]:
features = pd.DataFrame(features, columns=['eigenvalue_sum', 'omnivariance', 'eigenentropy', 'anisotropy', 'planarity', 'linearity', 'PCA1', 'PCA2', 'surface_variation', 'sphericity', 'verticality', 'nx', 'ny', 'nz'])

In [13]:
points_with_features = pd.concat([points, features], axis=1)
points_with_features = pd.concat([points_with_features, classes], axis=1)
points_with_features

,x,y,z,eigenvalue_sum,omnivariance,eigenentropy,anisotropy,planarity,linearity,PCA1,PCA2,surface_variation,sphericity,verticality,nx,ny,nz,label
0,289.110962,285.148987,137.710007,0.015525,0.002626,0.076370,0.964615,0.580688,0.383926,0.605525,0.373048,0.021427,0.035385,0.803205,-0.977368,-0.077607,-0.196795,6.000000
1,289.234009,285.190002,136.304001,0.025169,0.003449,0.111052,0.981330,0.682223,0.299107,0.581543,0.407600,0.010857,0.018670,0.949523,-0.937970,-0.343023,-0.050477,6.000000
2,289.177979,285.318970,136.171005,0.022403,0.003800,0.100914,0.964066,0.378544,0.585522,0.689459,0.285766,0.024775,0.035934,0.951077,-0.819375,-0.571167,-0.048923,6.000000
3,289.161987,285.018982,137.479996,0.014632,0.002272,0.072580,0.973096,0.578785,0.394311,0.612523,0.370998,0.016479,0.026904,0.983630,-0.987735,-0.155278,0.016369,6.000000
4,289.143982,285.171021,137.345993,0.026439,0.002632,0.110437,0.992285,0.253506,0.738779,0.788062,0.205858,0.006080,0.007715,0.950888,-0.995396,-0.082306,-0.049112,6.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6747643,356.797974,124.692017,137.091003,0.020476,0.005113,0.097486,0.863835,0.376551,0.487285,0.606472,0.310948,0.082580,0.136165,0.949365,-0.978583,-0.199529,0.050635,3.000000
6747644,356.780029,124.747009,137.039993,0.026006,0.004206,0.114020,0.969622,0.529914,0.439708,0.628666,0.352237,0.019098,0.030378,0.858457,0.912905,0.382844,-0.141543,3.000000
6747645,356.742981,124.804016,136.988007,0.034059,0.005073,0.138726,0.976537,0.433324,0.543213,0.675561,0.308588,0.015851,0.023463,0.862066,0.929421,0.342273,-0.137934,3.000000
6747646,356.690979,124.916992,136.884995,0.025921,0.004501,0.114394,0.961451,0.582674,0.378777,0.602493,0.374282,0.023225,0.038549,0.772900,-0.960005,-0.163754,0.227100,3.000000


In [14]:
points_with_features.to_csv(f'ProcessedData/features_{fileName}.csv')
print("DONE!")

DONE!


In [3]:
"""
csvFiles = ["ProcessedData/UncertaintyAdded/fullFeatures_Lille1_1.csv", "ProcessedData/UncertaintyAdded/fullFeatures_Lille1_2.csv", "ProcessedData/UncertaintyAdded/fullFeatures_Lille2.csv", "ProcessedData/UncertaintyAdded/fullFeatures_Paris.csv"]

with open("ProcessedData/UncertaintyAdded/fullFeatures_ParisLilleFullMergedData.csv", "w") as outfile:
    for idx, filePath in enumerate(csvFiles):
        with open(filePath, "r") as infile:
            lines = infile.readlines()
            if idx == 0:
                # Write entire first file (including header)
                outfile.writelines(lines)
            else:
                # Skip the header line and write the rest
                outfile.writelines(lines[1:])

print("Done")
"""

Done
